In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [3]:
def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [4]:
SYSTEM_PROMPT_8="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [5]:
SELF_REF="""
Given a test scenario and test cases to test that given test scenario.
Mark if test case can test the given test scenario or not through test steps, expected output, objective (although if it test other use case path or flow, if it is not used to test the given test scenario.)
And give explanation why you think the resulted test case reflect the given test scenario or not. 
Your response should keep the format of the inserted test cases.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "reflect": "yes/no",
    "explanation": "explain why you think this test case reflect the given test scenario or not",
  ],
}"""

In [6]:
scenarios = ["Successful Rating Submission", "Rating Submission with Added Comment",
             "Rating Submission with Added Picture", "Rating Submission with Added Video",
             "Rating Submission with Modified Star Ratings", "User Adjusts Quality Rating Before Submitting",
             "User Adjusts Seller Rating Before Submitting", "Adjust Shipping Rating Before Submitting",
             "User Adds Multiple Pictures Up to Maximum Limit"]

In [7]:
prompt_uc = """
Rating a product
Objective: This use case helps user to rate a product's quality for other users to know.
Actor: Users

Preconditions: User must have loged in to be able to access to tracking order status page. The product user want to rate must be in an order with status "Finished". User is in the tracking order status page at the tab Finished.

Main flow:
Step 1: User clicks on the Rating button of the product user wants to rate in the Finished order.
Step 2: System shows a pop up with rating form including the instruction of how to leave a good rating for receiving promotions from the platform, name of the product, the quality rating uses star rating system automatically tick 5 star by the system, there are comment section for user to leave a comment, an add picture button and an add video button. There are also Service rating section with seller rating and shipping rating both use star rating system automatically tick 5 star by the system.
Step 3: User clicks Done button.
Step 4: System saves and shows user rating in the product's detail page's rating section.

Alternative flow 1: User adjust quality rating point for product
At step 3 of the basic flow: User clicks on how many star they want to give in the quality rating section. There are no option to choose 0 star.
Go back to step 3 in the basic flow and continue with the steps from step 3.

Alternative flow 2: User adjust seller rating point for service rating section
At step 3 of the basic flow: User clicks on how many star they want to give in the Service rating section for seller's rating. There are no option to choose 0 star.
Go back to step 3 in the basic flow and continue with the steps from step 3.

Alternative flow 3: User adjust shipping rating point for service rating section
At step 3 of the basic flow: User clicks on how many star they want to give in the Service rating section for shipping's rating. There are no option to choose 0 star.
Go back to step 3 in the basic flow and continue with the steps from step 3.

Alternative flow 4: User leave a comment
At step 3 of the basic flow: User leaves a comment in the comment box.
Go back to step 3 in the basic flow and continue with the steps from step 3.

Alternative flow 5: User add a picture
At step 3 of the basic flow: User clicks on the Add picture button.
Step 4: A pop up shows up for user to choose a file. 
Step 5: User clicks on an image file. The files must be an image file, if user choose a file that is not an image, the system will not save that file.
Step 6: System shows the image chosen. The add image button and add video button are now change to a camera icon with 4/5 indicates maximum image can be add is 5 and a video icon with 0/1 indicates maximum image can be add is 1.
Go back to step 3 in the basic flow and continue with the steps from step 3.

Alternative flow 6: User add multiple pictures
At step 3 of the basic flow: User clicks on the Add picture button.
Step 4: A pop up shows up for user to choose a file. 
Step 5: User clicks on an image file. The files must be an image file, if user choose a file that is not an image, the system will not save that file.
Step 6: System shows the image chosen. The add image button and add video button are now change to a camera icon with 4/5 indicates maximum image can be add is 5 and a video icon with 0/1 indicates maximum image can be add is 1.
Step 7: User clicks on the camera icon.
Step 8: A pop up shows up for user to choose a file.
Step 9: User clicks on an image file.
Step 10: System shows the image chosen next to the previous chosen image.
Step 11: Repeat step 7 to 10 until it reaches the quantity of images user want or reaches maximum 5 of the system.
Go back to step 3 in the basic flow and continue with the steps from step 3.

Alternative flow 7: User add a video
At step 3 of the basic flow: User clicks on the Add video button.
Step 4: A pop up shows up for user to choose a file. 
Step 5: User clicks on a video file. The files must be a video file, if user choose a file that is not a video, the system will not save that file.
Step 6: System shows the video chosen. The add image button and add video button are now change to a camera icon with 0/5 indicates maximum image can be add is 5 and a video icon with 1/1 indicates maximum image can be add is 1.
Go back to step 3 in the basic flow and continue with the steps from step 3.
"""

In [8]:
for scenario in scenarios:
    i = 1
    while(i<=3):
        try: 
            promptTestCase = [
            { "role": "system", "content": SYSTEM_PROMPT_8},
            { "role": "user", "content": scenario + "\n" + prompt_uc}
            ]
            gpt_response = ask(promptTestCase, client, model)
            json_data = json.loads(gpt_response)
            write_json_to_txt_raw(json_data, scenario + "-Raw" + str(i) + ".txt")

            promptSelfRef = [
                { "role": "system", "content": SELF_REF},
                { "role": "user", "content": scenario + "\n" + gpt_response}
            ]
            final_response = ask(promptSelfRef, client, model)
            json_fin = json.loads(final_response)
            write_json_to_txt(json_fin, scenario + str(i) + ".txt")
            i+=1
        except Exception as e:
            print(f"An error occurred with scenario '{scenario}' iteration {i}: {e}")